In [1]:
import numpy as np
from fractions import Fraction # so that numbers are not displayed in decimal. 

In [2]:
def gen_problem(n_var, n_contrain):
    
    contrain = np.random.randint(low=-9, high=13, size=(n_var,n_contrain))
    bacis = np.eye(n_contrain)

    # A will contain the coefficients of the constraints 
    A = np.vstack((contrain,bacis)).T

    # b will contain the amount of resources 
    b = np.random.randint(low=-9, high=13, size=(n_contrain,))


    # c will contain coefficients of objective function Z
    cz = np.random.randint(low=-9, high=13, size=(n_var,))
    cb = np.zeros((n_contrain,))
    c = np.concatenate([cz,cb])
    
    B = np.arange(n_var,n_var + n_contrain)[np.newaxis].T
    n = np.arange(0,n_var)[np.newaxis].T
    
    return A, b, c, B, n

In [3]:
A, b, c, B, n = gen_problem(3,2)

In [4]:
# inputs 

# A will contain the coefficients of the constraints 
A = np.array([[-1,-1, -1, 1, 0],
              [2, -1,  1, 0, 1]])


# b will contain the amount of resources 
b = np.array([-2, 1])


# c will contain coefficients of objective function Z 
c = np.array([2, -6, 0, 0, 0])

B = np.array([[3], [4]])
n = np.array([[0], [1], [2]])

In [5]:
c

array([ 2, -6,  0,  0,  0])

In [6]:
# init parameter
xb = np.transpose([b]) 
zn = -c[n]
count = 0

In [7]:
if False not in (zn > 0):
    print('ok')

In [8]:
def dual_simplex(A, B, n, xb, zn, verbor=False):

    count = 0
    Bi = A[:,B].reshape((-1,len(B)))
    N = A[:,n].reshape((-1, len(n)))

    if verbor:
        A_hat = np.concatenate([B.T,xb.T,N.T,Bi.T]).T
        print("Dictionary\n", A_hat)
    
    while(np.min(xb)<0):
        i = np.argmin(xb)
        e = np.zeros((1,len(xb))).T
        e[i] = 1
        
        delta_zn = -(np.linalg.inv(Bi).dot(N)).T.dot(e)
    
        s = np.max(delta_zn/zn)**-1

        j = np.argmax(delta_zn/zn)
        e = np.zeros((1,len(zn))).T
        e[j] = 1

        delta_xb = np.linalg.inv(Bi).dot(N).dot(e)

        t = xb[i]/delta_xb[i]

        xb = xb - t*delta_xb
        zn = zn - s*delta_zn

        xb[i] = t
        zn[j] = s

        # pivot
        pivot = B[i].copy()
        B[i] = n[j].copy()
        n[j] = pivot

        Bi = A[:,B].reshape((-1,len(B)))
        N = A[:,n].reshape((-1, len(n)))

        A_hat = np.concatenate([B.T,xb.T,N.T,Bi.T]).T

        count += 1
        optimal = xb.T.dot(c[B]).reshape(-1)[0]

        if verbor:
            A_hat = np.concatenate([B.T,xb.T,N.T,Bi.T]).T
            print("iter:", count)
            print("Dictionary\n", A_hat)
            print("optimal:", optimal)
    
    sol = np.zeros(len(c))
    sol[B] = xb
    
    return {
        "iter": count,
        "optimal": optimal,
        "sol": sol
    }

In [9]:
dual_simplex(A.copy(), B.copy(), n.copy(), xb.copy(), zn.copy(), verbor=False)

/home/abelsliver/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in true_divide
/home/abelsliver/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide


{'iter': 3, 'optimal': -3.0, 'sol': array([0. , 0.5, 1.5, 0. , 0. ])}